## Solve Gridworld with policy iteration

In [1]:
# import packages
from Gridworld import Gridworld
import numpy as np

In [2]:
# function returns current position of player as coordinates
def find_position_player(array_playing_field):
    player = array_playing_field[0]
    for i in range(len(player)):
        if max(player[i])==1:
            x = i
    for j in range(len(player)):
        if player[x][j]==1:
            y = j
    return (x,y)

In [3]:
# function returns the instant reward given a position
def instant_reward(array, position):
    wall = array[3]
    pit = array[2]
    goal = array[1]

    first = position[0]
    second = position[1]

    if goal[first, second] == 1:
        return 10
    elif wall[first, second] == 1:
        return -50
    elif pit[first, second] == 1:
        return -100
    else:
        return -1

In [4]:
# function calculates the reward given a position and a move
def return_reward (array, position, move, grid):
    wall = array[3]
    pit = array[2]
    goal = array[1]

    if (position[0]== 0) & (move =='u'):
        return (-1, position[grid -1], position[1])
    elif (position[1]== 0) & (move =='l'):
        return (-1, position[0], position[1])
    elif (position[0]== 3) & (move =='d'):
        return (-1, position[0], position[1])
    elif (position[1]== grid -1) & (move =='r'):
        return (-1, position[0], position[1])
    else:
        first = position[0]
        second = position[1]
        if move == 'u':
            first = position[0] - 1
        elif move == 'd':
            first =  position[0] + 1
        elif move == 'r':
            second = position[1] + 1
        elif move == 'l':
            second = position[1] - 1
        else:
            return 'error'
        
        inst_reward = instant_reward(array,(first, second)) 

        return (inst_reward, first, second)

In [5]:
# function return the value from q-mat based on position and move
def get_q_value (q_mat, position, move, grid):
  
    # if move would take player off the board
    if (position[0]== 0) & (move =='u'):
        return q_mat[position[0], position[1]]
    elif (position[1]== 0) & (move =='l'):
        return q_mat[position[0], position[1]]
    elif (position[0]== grid -1) & (move =='d'):
        return q_mat[position[0], position[1]]
    elif (position[1]== grid -1) & (move =='r'):
        return q_mat[position[0], position[1]]
    
    # actual move
    else:
        first = position[0]
        second = position[1]
        if move == 'u':
            first = position[0] - 1
        elif move == 'd':
            first =  position[0] + 1
        elif move == 'r':
            second = position[1] + 1
        elif move == 'l':
            second = position[1] - 1
        else:
            return 'error'


        return q_mat[first, second]

In [6]:
# function calculates the best policies from position
def update_prob_vector(q_mat, prob_vector, position, grid):
    moves = ['u','r','d','l']
    store_v_next = []
    for move in moves:
        store_v_next.append(get_q_value( q_mat, position, move, grid=grid)) # only the first element is the reward
    max = np.max(store_v_next)
    # seach for max in vector and count how often it appears
    prob_vector = np.zeros(grid)
    for i in range(len(store_v_next)):
        if store_v_next[i] == max:
            prob_vector[i] = 1
      
    
    prob_vector /= prob_vector.sum()
    return prob_vector

In [7]:
# calculates the value of being at position based on what is possible in the next state
def exp_val_next_state(q_mat, position, prob_vector, grid):
    moves = ['u','r','d','l']
    expected_value = 0
    for i in range(len(moves)):
        move = moves[i]
        expected_value += prob_vector[i] * get_q_value(q_mat, position, move, grid=grid)
    return expected_value

In [8]:
def create_mat_policy_vec(dimension, grid):
    prob_vector = np.ones(4)
    prob_vector /= prob_vector.sum()
    prob_vector

    # set up a matrix storing for each position the policy vector
    mat_of_prob_vec = []
    for i in range (grid):
        temp = []
        for j in range (grid):
            temp.append(prob_vector)
        mat_of_prob_vec.append(temp)

    return mat_of_prob_vec

In [9]:
# instantiate a random game and show board
grid = 4
game = Gridworld(size=grid, mode='random')
print(game.display())
print('')

[[' ' ' ' ' ' ' ']
 [' ' '-' ' ' ' ']
 ['+' ' ' 'W' ' ']
 [' ' ' ' 'P' ' ']]



In [10]:
# instantiate a random game and try to solve it within 10 moves

grid = 4
game = Gridworld(size=grid, mode='random')
print(game.display())
print('')

# define possible moves
moves = ['u','r','d','l']

array_playing_field = game.board.render_np()

gamma = 0.9 # discount factor
q_mat = np.zeros((grid,grid)) # q_mat initialize to zero
mat_policy_vec = create_mat_policy_vec(dimension=grid, grid=grid) # matrix with initialized policy vector for each position

q_mat_next = np.zeros((grid,grid)) # q_mat for the next state initialized to zero

# loops to update q_mat and policies
counter = 0
while counter < 10:

    # update q_mat
    for i in range(grid):
        for j in range(grid):
            # value is sum of instant reward plus expected value
            q_mat_next[i][j] = instant_reward(array = array_playing_field, position=(i,j)) + gamma * exp_val_next_state(q_mat=q_mat, position=(i,j), prob_vector=mat_policy_vec[i][j], grid=grid) # Bellman equation

    #print(q_mat_next)
    # update mat_prob_vec
    for i in range(grid):
        for j in range(grid):
            mat_policy_vec[i][j] = update_prob_vector(q_mat=q_mat_next, prob_vector=mat_policy_vec[i][j], position=(i,j), grid=grid)
    
    # update q_mat
    q_mat = q_mat_next.copy()
    counter += 1

list_of_moves = []
while game.reward()==-1:
    position = find_position_player(array_playing_field=game.board.render_np())
    print('position: ',position)
    move_idx = np.argmax(mat_policy_vec[position[0]][position[1]])
    list_of_moves.append(moves[move_idx])
    game.makeMove(moves[move_idx])

if game.reward() == 10:
    print('game won!')
    
    

[[' ' ' ' 'P' ' ']
 [' ' 'W' ' ' ' ']
 [' ' ' ' '-' ' ']
 [' ' ' ' '+' ' ']]

position:  (0, 2)
position:  (0, 3)
position:  (1, 3)
position:  (2, 3)
position:  (3, 3)
game won!


In [11]:
# show the moves
list_of_moves

['r', 'd', 'd', 'd', 'l']